In [ ]:
import sys
import os

# Get the absolute path of the parent directory (project root)
project_root = os.path.abspath('..')

# Add it to sys.path if it's not already there
if project_root not in sys.path:
    sys.path.append(project_root)
    
from bsort.config import settings

In [ ]:
from ultralytics import settings as ultralytics_settings
ultralytics_settings.update({
    'datasets_dir': os.path.join(project_root, 'datasets'),
    'weights_dir': os.path.join(project_root, 'weights'),
    'runs_dir': os.path.join(project_root, 'runs'),
    'mlflow': False,
    'wandb': True
})

import wandb
wandb.init()

In [ ]:
from bsort.utils import extract_and_prepare_dataset, print_metrics, download_roboflow_dataset
from ultralytics import YOLO
from pathlib import Path

In [ ]:
PROJECT_NAME = "bottle_cap_project"
RUN_NAME_PRETRAIN = "stage1_public_pretrain"
RUN_NAME_FINETUNE = "stage2_sample_finetune"

In [ ]:
target_location = Path("..") / "datasets" / "processed" / "public"
public_config_path = target_location / "data.yaml"

download_roboflow_dataset(
    workspace="work3-dqzz5",
    project="bottle-cap-y6pzg",
    version_number=1,
    location=target_location
)

In [ ]:
zip_file_path = Path("..") / "datasets" / "sample.zip"
dataset_root_path = Path("..") / "datasets"

sample_config_path = extract_and_prepare_dataset(
    zip_path=zip_file_path,
    root_dir=dataset_root_path
)

In [ ]:
model_stage1 = YOLO("yolo11p.yaml")

results_stage1 = model_stage1.train(
    data=str(public_config_path),
    project=PROJECT_NAME,
    name=RUN_NAME_PRETRAIN,
    epochs=25,
    verbose=True,
    # Augmentation settings
    hsv_h=0.0,
    hsv_s=0.25,
    hsv_v=0.0,
    mixup=0.0,
    optimizer="SGD",
    lr0=0.01,
    weight_decay=0.01,
    fliplr=0.5,
    mosaic=0.0
)

In [ ]:
stage1_weights_path =  Path(PROJECT_NAME) / RUN_NAME_PRETRAIN / "weights" / "best.pt"
model_finetune = YOLO(stage1_weights_path)
results_finetune = model_finetune.train(
    data=str(sample_config_path),
    project=PROJECT_NAME,
    name=RUN_NAME_FINETUNE,
    epochs=150,
    verbose=True,

    hsv_h=0.0,
    hsv_s=0.25,
    hsv_v=0.25,
    mixup=0.0,
    optimizer="SGD",
    lr0=0.02,
    weight_decay=0.01,
    fliplr=0.5,
    mosaic=0.25
)

In [ ]:
print("Best model results")
print_metrics(results_finetune)